## Rules
1. Four coloured team
2. Each team must be covered by 1 person everyday
* On weekdays, one employee can cover 2 teams
* On weekends, employees can only be on one team, but can cover 2 teams if in gold
* One cannot do both Red and Blue on weekends
* An employee cannot work for 3 weekends in a row
* If an employee is on call for 2 weekends in a row (Sat + Sun), they must have the subsequent Fri-Sun off
* Different days of the week have score. More the score, less ideal to work on:

| Saturday |Sunday | Tuesday |Thursday |Wednesday |Monday | Friday |
|----------|-------|---------|---------|----------|-------|--------|
| 5        |   4   |    3    |    2    |    2     |    1  |   1    |
        
* Design the algorithm to find the best schedule based on the lowest overall total
* Some employees will request days off, they should not be penaltized to have more work

In [1]:
import random
import calendar
import pandas as pd
import numpy as np
from numpy.random import choice
import timeit
import csv
import warnings
import datetime
from datetime import date, timedelta

# warnings.filterwarnings("ignore")
# Read from CSV or Not (True = read, False = Random)
read_from_csv = True

if read_from_csv:
    global cycle_start_date 
    global cycle_no_of_days 
    global no_of_empl 
    
    cycle_start_date= "May 06, 2019"
    cycle_no_of_days= 28
    no_of_empl= 11

#### Get year, month, no of days

In [2]:
def get_dates():
    if read_from_csv:
        global cycle_start_date
        global cycle_no_of_days
        start_date = datetime.datetime.strptime(cycle_start_date, "%b %d, %Y").date()
        return start_date.year,start_date.month, start_date.day, cycle_no_of_days
    else:
        year = random.randint(1970,2100)
        month = random.randint(1,12)
        start_day, no_of_days = calendar.monthrange(year, month)
        return year, month, 1, no_of_days

In [3]:
if read_from_csv:
    file = pd.read_csv("call_schedule.csv", usecols = [0,1,2,3,4,5,6,7], nrows = no_of_empl)
    display(file)

,Employees,Team Colour,Off-Service,Worked the weekend before?,Dates requested to WORK,Dates requested to not work,Formal days off,Max shifts in 28 days
0,Bloom,Blue,0,0,NaN,May 18-26,7,5
1,Shahmoradi,Blue,0,0,May 29,May 18-19\nMay 22-23\nMay 31-June 2,1,6
2,Postnikoff,Blue,1,0,NaN,May 10-13\nMay 25-June 2,11,4
3,Gordon,Red,0,0,NaN,NaN,0,7
4,Appelton,Red,0,0,NaN,May 18-26\nJune 1,7,5
5,Lange,Red,1,0,NaN,May 6-12\nJune 1-2\nMay 19\nMay 26,7,5
6,Head,Red,0,1,NaN,May 25-June 2,7,5
7,Oro,Silver,0,0,NaN,May 11-12,0,7
8,Verity,Silver,1,0,NaN,May 15-21,6,5
9,Swami,Silver,0,0,NaN,NaN,0,7


#### Create month table

In [4]:
def get_score(weekday_name):
    day_score = []
    for name in weekday_name:
        if name == 'Saturday' :
            day_score.append(5)
        elif name == 'Friday':
            day_score.append(4)
        elif name == 'Sunday':
            day_score.append(3)
        elif name == 'Tuesday' or name =='Thursday' :
            day_score.append(2)
        else:
            day_score.append(1)
    return day_score

def create_table(year, month, day, no_of_days):
    
    
    start_date = datetime.datetime.strptime(str(month)+str(day)+str(year), "%m%d%Y").date()
    
    days = []
    for i in range(no_of_days):
        days.append(start_date + timedelta(days=i))
    
    df = pd.DataFrame({'Date': days})
    
    weekday_names = {6:'Sunday', 0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday'}
    Day = [weekday_names[x.weekday()] for x in df.Date]
    df['Day'] = Day
    df['Score'] = get_score(Day)
    df['Blue'] = 'NA'
    df['Red'] = 'NA'
    df['Silver'] = 'NA'
    df['Gold'] = 'NA'
    return df

#### List of random no. of employees
#### Also select last 2 weekend's work randomly 

In [5]:
def get_dates_from_string(str_dates):
    days_leave = []
    num_days_leave = 0
    for date in str_dates.split('\n'):
        start_month = date.split('-')[0].split(' ')[0][:3]
        start_day = date.split('-')[0].split(' ')[1]
        start_date = datetime.datetime.strptime(start_month+start_day+str(year), "%b%d%Y").date()

        if len(date.split('-'))==1:
            end_month = start_month
            end_day = start_day
        else:
            if len(date.split('-')[1].split(' '))==1:
                end_month = start_month
                end_day = date.split('-')[1]
            else:
                end_month = date.split('-')[1].split(' ')[0][:3]
                end_day = date.split('-')[1].split(' ')[1]
        end_date = datetime.datetime.strptime(end_month+end_day+str(year), "%b%d%Y").date()
        delta = end_date - start_date
        num_days_leave += delta.days
        for i in range(delta.days + 1):
            days_leave.append(start_date + timedelta(days=i))
            
    return days_leave, num_days_leave

In [6]:
def create_employee_list():
    global no_of_empl
    global no_of_days
    if not read_from_csv:
        no_of_empl = random.randint(11,11)
        #6 random employees worked 2 weekends back
        weekend_1 = random.sample(range(1,no_of_empl+1), 6)
        weekend_2 = random.sample(range(1,no_of_empl+1), 6)
    else:
        file = pd.read_csv("call_schedule.csv", usecols = [0,1,2,3,4,5,6,7], nrows = no_of_empl)
        weekend_2 = list(file['Off-Service'])
        weekend_1 = list(file['Worked the weekend before?'])

    empl_list = []
    for i in range(1,no_of_empl+1):
        if not read_from_csv:
            one_weekend_back = 0
            two_weekend_back = 0
            if i in weekend_1:
                one_weekend_back = 1
            if i in weekend_2:
                two_weekend_back = 1
            if(i<10):
                empl_list.append(['emp_0' + str(i), 0, two_weekend_back, one_weekend_back, 0])
            else:
                empl_list.append(['emp_' + str(i), 0, two_weekend_back, one_weekend_back, 0])
        else:
            empl_list.append([(file['Employees'])[i-1], 0, \
                              weekend_2[i-1], weekend_1[i-1], 0])
            
    if not read_from_csv:
        #Random no. of employee on holiday for random amount of days
        #5-10 employees taking a leave
        no_of_empl_leave = random.randint(5,10)
        
        #selecting random employees who are takeing a leave
        empl_list_leave = random.sample(range(0,no_of_empl+1), no_of_empl_leave)

        #assiging random amount of leave days (max 10)
        for loc,empl in enumerate(empl_list):
            if loc in empl_list_leave:
                empl[4] = random.randint(1,3)

        for num,empl in enumerate(empl_list):
            days_leave = random.sample(list(df_teams['Date']), empl[4])
            empl.append(days_leave)
            empl.append(10000)
            empl.append(no_of_days/(no_of_days-empl[4]))
            empl.append(100000)
            empl.append(0)
    else:
        for num, str_dates in enumerate(file['Dates requested to not work']):
            str_dates = str(str_dates)
            days_leave = []
            num_days_leave = 0
            if not str_dates == 'nan':
                days_leave, num_days_leave = get_dates_from_string(str_dates)
            empl_list[num][4] = num_days_leave
            empl_list[num].append(days_leave)
            empl_list[num].append(10000)
            empl_list[num].append(no_of_days/(no_of_days-empl_list[num][4]))
            empl_list[num].append(file['Max shifts in 28 days'][num])
            empl_list[num].append(0)
    
#     empl_list =  "[Employee name, 
#     If working this week, 
#     If worked one weekend back, 
#     if worked two weekends back, 
#     No of days on leave, 
#     list of days on leave,
#     Weight, 
#     Score Multiplier,
#     Max shifts in 28 days,
#     no of days worked currently]"

    return empl_list, no_of_empl

In [7]:
def employee_schedule(year, month, empl_list):
    start_date = str(year) + '-' + str(month) +'-01'
    end_date = str(year) + '-' + str(month) +'-'+str(no_of_days)
    df = pd.DataFrame({'Date': pd.date_range(start_date, end_date)})
    for emp in empl_list:
        df[emp[0]] = 0
    df.loc['Total'] = 0.0001
    return df

#### Randomly assigning work to employees

In [8]:
def update_df_emp(teams, empl_choice, loc, score):
    for empl in empl_list:
        if empl == empl_choice:
            df_emp.loc[loc,empl[0]] += score*teams*empl[7]

def choose_empl(free_list, method='best'):
    if len(free_list) == 0:
        return ['NA']
    if method == 'random' :
        ## Select completly random employee for next day
        return random.choice(free_list)
    elif method == 'weighted' :
        ## Select an employee Psuedo-randomly (based on thier weights)
        totals = []
        running_total = 0

        for w in [empl[6] for empl in free_list]:
            running_total += w
            totals.append(running_total)

        rnd = random.random() * running_total
        for i, total in enumerate(totals):
            if rnd < total:
                return free_list[i]
    elif method == 'best' :
        ## Select employee who has lowest total score till date
        highest = -1
        best = []
        random.shuffle(free_list)

        for empl in free_list:
            if empl[6] > highest:
                highest = empl[6]
                best = empl

        return best
    else:
        return None
    
def get_ml_inputs(score_next_day, free_list, is_weekend, day):
    random.shuffle(free_list)
    input_to_ml = []
    input_to_ml.append(score_next_day)
    for empl in free_list:
        input_to_ml.append(df_emp.loc['Total'][empl[0]])
    
    score_mults = []
    for empl in free_list:
        input_to_ml.append(empl[7])

    input_to_ml.append(is_weekend)
    
    with open('test.txt','w') as f:
        f.write(str(input_to_ml))
    
    np.save(f'selection/{day}.npy', input_to_ml)

In [9]:
def update_empl_list(choice):
    for num,empl in enumerate(empl_list):
        if empl == choice:
            empl_list[num][9] += 1

In [10]:
def assign_schedule(method='best'):
    NA = False
    for loc, date in enumerate(df_teams['Date']):

        score = df_teams.loc[loc,'Score']
        
        #weekend
        if(date.weekday() == 5 or date.weekday() == 6):
            free_list = []
            for empl in empl_list:
                if ((empl[2] == 0 or empl[3] == 0) and empl[1] == 0) and (date not in empl[5]):
                    if not (empl[2] == 1 and empl [3] == 1 and date.weekday() == 6):
                        if empl[9] < empl[8]:
                            free_list.append(empl)
            gold_choice = choose_empl(free_list, method)
            df_teams.at[loc, 'Gold'] = gold_choice[0]

            free_list = [x for x in free_list if x != gold_choice]

            #assign same to one of the other three team (red,blue,silver)
            with_gold = choice(['Red','Blue','Silver'])
            df_teams.at[loc, with_gold] = gold_choice[0]
            update_df_emp(2, gold_choice, loc, score)

            #assign different employees to remaining 2 team
            other_choice_1 = choose_empl(free_list, method)
            free_list = [x for x in free_list if x != other_choice_1]
            other_choice_2 = choose_empl(free_list, method)
            if len(free_list) == 0:
                NA = True

            if with_gold == 'Blue':
                df_teams.at[loc, 'Red'] = other_choice_1[0]
                df_teams.at[loc, 'Silver'] = other_choice_2[0]
                update_df_emp(1,other_choice_1,loc, score)
                update_df_emp(1,other_choice_2,loc, score)
            elif with_gold == 'Red':
                df_teams.at[loc, 'Blue'] = other_choice_1[0]
                df_teams.at[loc, 'Silver'] = other_choice_2[0]
                update_df_emp(1,other_choice_1,loc, score)
                update_df_emp(1,other_choice_2,loc, score)
            else:
                df_teams.at[loc, 'Blue'] = other_choice_1[0]
                df_teams.at[loc, 'Red'] = other_choice_2[0]
                update_df_emp(1,other_choice_1,loc, score)
                update_df_emp(1,other_choice_2,loc, score)
            
            update_empl_list(gold_choice)
            update_empl_list(other_choice_1)
            update_empl_list(other_choice_2)

            #update weekend work status
            for i in range(len(empl_list)):
                if(empl_list[i]==gold_choice or empl_list[i]==other_choice_1 or empl_list[i]==other_choice_2):
                    empl_list[i][1] = 1

            if(date.weekday() == 6):
                for i in range(len(empl_list)):
                    empl_list[i][3] = empl_list[i][2]
                    empl_list[i][2] = empl_list[i][1]
                    empl_list[i][1] = 0
                    
    #   Weekday
        else:
            free_list = []
            for empl in empl_list:
                if date not in empl[5]:
                    if not (empl[2] == 1 and empl [3] == 1 and date.weekday() == 4):
                        if empl[9] < empl[8]:
                            free_list.append(empl)
            
            #Get random choice for Gold
            gold_choice = choose_empl(free_list, method)
            df_teams.at[loc, 'Gold'] = gold_choice[0]

            #assign same to one of the other three team (red,blue,silver)
            with_gold = choice(['Red','Blue','Silver'])
            df_teams.at[loc, with_gold] = gold_choice[0]

            update_df_emp(2, gold_choice, loc, score)
            free_list = [x for x in free_list if x != gold_choice]
            if len(free_list) == 0:
                NA = True

            #assign different employee to remaining 2 team
            other_choice = choose_empl(free_list, method)
            if with_gold == 'Blue':
                df_teams.at[loc, 'Red'] = other_choice[0]
                df_teams.at[loc, 'Silver'] = other_choice[0]
                update_df_emp(2,other_choice,loc, score)
            elif with_gold == 'Red':
                df_teams.at[loc, 'Blue'] = other_choice[0]
                df_teams.at[loc, 'Silver'] = other_choice[0]
                update_df_emp(2,other_choice,loc, score)
            else:
                df_teams.at[loc, 'Blue'] = other_choice[0]
                df_teams.at[loc, 'Red'] = other_choice[0]
                update_df_emp(2,other_choice,loc, score)
            
            update_empl_list(gold_choice)
            update_empl_list(other_choice)
            

        df_emp.loc['Total'] = 0.0001
        df_emp.loc['Total'] = df_emp.sum()
        for empl in empl_list:
            empl[6] = 1 / df_emp.loc['Total'][empl[0]]
    
#     df_teams = df_teams.sort_values(by=['Date'])

#     return df_teams, df_emp, empl_list
    return NA


In [11]:
# tosave = [no_of_empl, no_of_days, wordload_std, std_no_of_empl, score_to_minimize]
# with open('std.csv','a') as f:
#     writer = csv.writer(f)
#     writer.writerow(tosave)

In [39]:
minimum_score_with_NA = 9999
minimum_score_without_NA = 9999
df_teams_list = []

for i in range(100):
    
    year, month, day, no_of_days = get_dates()

    df_teams = create_table(year, month, day, no_of_days)

    empl_list, no_of_empl = create_employee_list()

    df_emp = employee_schedule(year, month, empl_list)
    df_teams_list.append(df_teams)

    method = 'best'
    NA = assign_schedule(method)
    
    workload_array = df_emp.loc['Total'][1:]
    for num,workload in enumerate(workload_array):
        workload *= no_of_days / (no_of_days-empl_list[num][4])
        workload_array[num] = workload
    wordload_std = np.std(workload_array)
    std_no_of_empl = wordload_std * np.sqrt(no_of_empl)
    score_to_minimize = wordload_std * np.sqrt(no_of_empl) / np.sqrt(no_of_days)
    
    if not NA and (minimum_score_without_NA > score_to_minimize):
        minimum_score_without_NA = score_to_minimize
        df_emp_best_without_NA = df_emp.copy(deep=True)
        df_teams_best_without_NA = df_teams.copy(deep=True)
        print("Minimum Score without NA:",minimum_score_without_NA)
    if NA and (minimum_score_with_NA > score_to_minimize):
        minimum_score_with_NA = score_to_minimize
        df_emp_best_with_NA = df_emp.copy(deep=True)
        df_teams_best_with_NA = df_teams.copy(deep=True)
        print("Minimum Score with NA:",minimum_score_with_NA)
        

Minimum Score with NA: 4.84080075096484
Minimum Score with NA: 4.469942721347617
Minimum Score with NA: 3.039068311206745
Minimum Score with NA: 2.9314877034536027
Minimum Score with NA: 2.738213437419107
Minimum Score with NA: 2.1759071464596156


In [13]:
file = pd.read_csv("call_schedule.csv", usecols = [0,1,2,3,4,5,6,7], nrows = no_of_empl)
display(file)

,Employees,Team Colour,Off-Service,Worked the weekend before?,Dates requested to WORK,Dates requested to not work,Formal days off,Max shifts in 28 days
0,Bloom,Blue,0,0,NaN,May 18-26,7,5
1,Shahmoradi,Blue,0,0,May 29,May 18-19\nMay 22-23\nMay 31-June 2,1,6
2,Postnikoff,Blue,1,0,NaN,May 10-13\nMay 25-June 2,11,4
3,Gordon,Red,0,0,NaN,NaN,0,7
4,Appelton,Red,0,0,NaN,May 18-26\nJune 1,7,5
5,Lange,Red,1,0,NaN,May 6-12\nJune 1-2\nMay 19\nMay 26,7,5
6,Head,Red,0,1,NaN,May 25-June 2,7,5
7,Oro,Silver,0,0,NaN,May 11-12,0,7
8,Verity,Silver,1,0,NaN,May 15-21,6,5
9,Swami,Silver,0,0,NaN,NaN,0,7


In [40]:
df_teams_best_with_NA

,Date,Day,Score,Blue,Red,Silver,Gold
0,2019-05-06,Monday,1,Oro,Bloom,Oro,Bloom
1,2019-05-07,Tuesday,2,Gordon,Postnikoff,Gordon,Postnikoff
2,2019-05-08,Wednesday,1,Head,Head,Appelton,Appelton
3,2019-05-09,Thursday,2,Shahmoradi,Verity,Verity,Shahmoradi
4,2019-05-10,Friday,4,Swami,Swami,Rashad,Rashad
5,2019-05-11,Saturday,5,Bloom,Head,Appelton,Bloom
6,2019-05-12,Sunday,3,Shahmoradi,Verity,Gordon,Gordon
7,2019-05-13,Monday,1,Oro,Oro,Lange,Lange
8,2019-05-14,Tuesday,2,Oro,Oro,Lange,Lange
9,2019-05-15,Wednesday,1,Postnikoff,Swami,Swami,Postnikoff


In [38]:
df_teams_list[random.randint(1,100)]

,Date,Day,Score,Blue,Red,Silver,Gold
0,2019-05-06,Monday,1,Swami,Swami,Head,Head
1,2019-05-07,Tuesday,2,Appelton,Bloom,Bloom,Appelton
2,2019-05-08,Wednesday,1,Gordon,Gordon,Rashad,Rashad
3,2019-05-09,Thursday,2,Postnikoff,Postnikoff,Shahmoradi,Shahmoradi
4,2019-05-10,Friday,4,Oro,Oro,Verity,Verity
5,2019-05-11,Saturday,5,Swami,Gordon,Rashad,Gordon
6,2019-05-12,Sunday,3,Head,Shahmoradi,Bloom,Head
7,2019-05-13,Monday,1,Appelton,Lange,Appelton,Lange
8,2019-05-14,Tuesday,2,Postnikoff,Postnikoff,Lange,Lange
9,2019-05-15,Wednesday,1,Rashad,Swami,Rashad,Swami


In [16]:
empl_list

[['Bloom',
  0,
  1,
  0,
  8,
  [datetime.date(2019, 5, 18),
   datetime.date(2019, 5, 19),
   datetime.date(2019, 5, 20),
   datetime.date(2019, 5, 21),
   datetime.date(2019, 5, 22),
   datetime.date(2019, 5, 23),
   datetime.date(2019, 5, 24),
   datetime.date(2019, 5, 25),
   datetime.date(2019, 5, 26)],
  0.025510139004747433,
  1.4,
  5,
  5],
 ['Shahmoradi',
  0,
  0,
  1,
  4,
  [datetime.date(2019, 5, 18),
   datetime.date(2019, 5, 19),
   datetime.date(2019, 5, 22),
   datetime.date(2019, 5, 23),
   datetime.date(2019, 5, 31),
   datetime.date(2019, 6, 1),
   datetime.date(2019, 6, 2)],
  0.032966924284864994,
  1.1666666666666667,
  6,
  5],
 ['Postnikoff',
  0,
  0,
  0,
  11,
  [datetime.date(2019, 5, 10),
   datetime.date(2019, 5, 11),
   datetime.date(2019, 5, 12),
   datetime.date(2019, 5, 13),
   datetime.date(2019, 5, 25),
   datetime.date(2019, 5, 26),
   datetime.date(2019, 5, 27),
   datetime.date(2019, 5, 28),
   datetime.date(2019, 5, 29),
   datetime.date(2019,

In [17]:
df_teams

,Date,Day,Score,Blue,Red,Silver,Gold
0,2019-05-06,Monday,1,Gordon,Rashad,Gordon,Rashad
1,2019-05-07,Tuesday,2,Appelton,Appelton,Postnikoff,Postnikoff
2,2019-05-08,Wednesday,1,Oro,Verity,Verity,Oro
3,2019-05-09,Thursday,2,Shahmoradi,Shahmoradi,Head,Head
4,2019-05-10,Friday,4,Swami,Bloom,Swami,Bloom
5,2019-05-11,Saturday,5,Gordon,Rashad,Verity,Gordon
6,2019-05-12,Sunday,3,Shahmoradi,Appelton,Head,Shahmoradi
7,2019-05-13,Monday,1,Oro,Oro,Lange,Lange
8,2019-05-14,Tuesday,2,Oro,Oro,Lange,Lange
9,2019-05-15,Wednesday,1,Postnikoff,Rashad,Rashad,Postnikoff
